# Milestone 2: Cryptography and Encryption Techniques
#### Ahmed Hassan 58-0671
#### Ziad Ekramy 58-6936

# Part 1 – Password Encryption

## Playfair Cipher Implementation

In [1]:
import string

In [2]:
def generate_playfair_matrix(key): # Build playfair matrix
    key = key.lower().replace("j", "i")
    seen = set()
    matrix = []

    for char in key:
        if char.isalpha() and char not in seen:
            seen.add(char)
            matrix.append(char)

    for char in string.ascii_lowercase:
        if char == "j":
            continue
        if char not in seen:
            seen.add(char)
            matrix.append(char)

    matrix_5x5 = [matrix[i*5:(i+1)*5] for i in range(5)] 
    return matrix_5x5


def find_position(matrix, char): # Locate row and column of a letter in the matrix
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                return i, j
    return None


def prepare_playfair_text(text): # Clean plaintext and convert it into pairs suitable for Playfair encryption
    text = text.lower().replace("j", "i")
    cleaned = [c for c in text if c.isalpha()]

    digraphs = []
    i = 0

    while i < len(cleaned):
        a = cleaned[i]
        if i + 1 < len(cleaned):
            b = cleaned[i + 1]
            if a == b:
                digraphs.append(a + "x")
                i += 1
            else:
                digraphs.append(a + b)
                i += 2
        else:
            digraphs.append(a + "x")
            i += 1
    return digraphs

In [3]:
def playfair_encrypt(plaintext, key): # Encrypting Playfair
    matrix = generate_playfair_matrix(key)
    digraphs = prepare_playfair_text(plaintext)
    ciphertext = ""

    for pair in digraphs:
        a, b = pair[0], pair[1]
        row1, col1 = find_position(matrix, a)
        row2, col2 = find_position(matrix, b)

        if row1 == row2:  # If same row
            ciphertext += matrix[row1][(col1 + 1) % 5]
            ciphertext += matrix[row2][(col2 + 1) % 5]

        elif col1 == col2:  # If same column
            ciphertext += matrix[(row1 + 1) % 5][col1]
            ciphertext += matrix[(row2 + 1) % 5][col2]

        else:  # If not same column / row
            ciphertext += matrix[row1][col2]
            ciphertext += matrix[row2][col1]

    return ciphertext

In [4]:
def playfair_decrypt(ciphertext, key): # Decrypting Playfair
    matrix = generate_playfair_matrix(key)
    plaintext = ""

    
    digraphs = [ciphertext[i:i+2] for i in range(0, len(ciphertext), 2)] # Split ciphertext into pairs

    for pair in digraphs:
        a, b = pair[0], pair[1]
        row1, col1 = find_position(matrix, a)
        row2, col2 = find_position(matrix, b)

        if row1 == row2:  # If same row → shift left
            plaintext += matrix[row1][(col1 - 1) % 5]
            plaintext += matrix[row2][(col2 - 1) % 5]

        elif col1 == col2:  # If same column → shift up
            plaintext += matrix[(row1 - 1) % 5][col1]
            plaintext += matrix[(row2 - 1) % 5][col2]

        else:  # If not same column / row
            plaintext += matrix[row1][col2]
            plaintext += matrix[row2][col1]

    return plaintext

In [5]:
playfair_encrypt("meet me after noon", "LETTER")

'irtrirrgrtamqvuh'

In [6]:
playfair_decrypt("irtrirrgrtamqvuh", "LETTER")

'meetmeafternoxon'

## Vigenère Cipher Implementation

In [7]:
def vigenere_encrypt(plaintext, key): # Encrypting Vigenère
    plaintext = ''.join([c.lower() for c in plaintext if c.isalpha()])
    key = key.lower()
    ciphertext = ""
    key_index = 0

    for char in plaintext:
        shift = ord(key[key_index % len(key)]) - ord('a')
        encrypted_char = chr(((ord(char) - ord('a') + shift) % 26) + ord('a'))
        ciphertext += encrypted_char
        key_index += 1

    return ciphertext

In [8]:
def vigenere_decrypt(ciphertext, key): # Decrypting Vigenère
    key = key.lower()
    plaintext = ""
    key_index = 0

    for char in ciphertext:
        shift = ord(key[key_index % len(key)]) - ord('a')
        decrypted_char = chr(((ord(char) - ord('a') - shift) % 26) + ord('a'))
        plaintext += decrypted_char
        key_index += 1

    return plaintext 

In [9]:
vigenere_encrypt("testmycodeforthisprojectthatispartofmyassignedtasksforthissemester", "mcqueen")

'fginqcpafuzsvgtkijvswqejnleguufuvxbroouwwvspuxxefwuvivxuuuiyqiffgh'

In [10]:
vigenere_decrypt("fginqcpafuzsvgtkijvswqejnleguufuvxbroouwwvspuxxefwuvivxuuuiyqiffgh","mcqueen")

'testmycodeforthisprojectthatispartofmyassignedtasksforthissemester'

## Password Based Key Derivation Function 2 (PBKDF2) Implementation

In [11]:
import hashlib
import secrets
import base64
import hmac
import time


DEFAULT_ITERATIONS = 120_000   # Work factor
SALT_BYTES = 16               # 16-byte random salt
DK_LENGTH = 32                # Key length: 32 bytes
HASH_NAME = "sha256"          # PBKDF2 PRF = HMAC-SHA256

# Here we call the imports we need & define default parameters and global configurations

In [12]:
def _b64encode(b: bytes) -> str:
    return base64.b64encode(b).decode("ascii")

def _b64decode(s: str) -> bytes:
    return base64.b64decode(s.encode("ascii"))

# Here we encode and decode by turning bytes to string & string to bytes

In [13]:
def pbkdf2_hash_password(
    password: str,
    iterations: int = DEFAULT_ITERATIONS,
    salt: bytes | None = None,
    dklen: int = DK_LENGTH,
    hash_name: str = HASH_NAME
) -> str:
  

    if salt is None:     
        salt = secrets.token_bytes(SALT_BYTES) # Generate random salt if not provided

    password_bytes = password.encode("utf-8")

   
    dk = hashlib.pbkdf2_hmac(hash_name, password_bytes, salt, iterations, dklen)  # Derive the key

    
    parts = [
        f"pbkdf2_{hash_name}",
        str(iterations),
        _b64encode(salt),
        _b64encode(dk)
    ] # Formatting 
    return "$".join(parts)

# Here we implement our Function

### Test PBKDF2 

In [14]:
password = "Ah!eb@2004_761!!"

password = pbkdf2_hash_password(password) # Generate hash
print("Generated Hash:", password)

Generated Hash: pbkdf2_sha256$120000$JyHk5oAx/rauMGIKcuGcUg==$4vTEDuUMl5CinwoOlSahgEZE7jp6bBwQGjWHtQz73F4=
